In [37]:
import cvxpy as cp
import numpy as np

# Generate a random non-trivial linear program.
#d = 10
m = 2
np.random.seed(1)
#s0 = np.random.randn(d)
#lamb0 = np.maximum(-s0, 0)
#s0 = np.maximum(s0, 0)
#rand_z = np.random.randn(m)
#A = np.random.randn(d, m) # d constraints, m decision vars
#b = A @ rand_z + s0
#c = -A.T @ lamb0

b0 = np.random.randn(m)
b1 = b0 + np.random.exponential(m)
y = np.random.randn(m)



# Define and solve the CVXPY problem.
z = cp.Variable(m)
#prob = cp.Problem(cp.Minimize(c.T@z),
#                 [A @ z <= b])
prob = cp.Problem(cp.Minimize(y.T@z),
                 [b0 <= z, z <= b1])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution z is")
print(z.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)

delta = .01 # how to choose epsilon? maybe compute from within function?

def fd(x, y, model):
    sens = torch.zeros(len(x))
    y_hat = model(x).detach().numpy()
    #print(c_hat)
    for i in range(len(y_hat)):
        z = cp.Variable(m)
        #prob = cp.Problem(cp.Minimize(y_hat[i,:].T@z),
                 #[A @ z <= b])
        prob = cp.Problem(cp.Minimize(y_hat[i,:].T@z),
                 [b0 <= z, z <= b1])
        prob.solve()
        #print("\nThe optimal value is", prob.value)
        #print("A solution x is")
        #print(z.value)
        #print("A dual solution is")
        #print(prob.constraints[0].dual_value)
        z0 = z.value
        #print(z0.shape)
        sens[i] = 2*delta*np.linalg.norm(z0, np.inf)
    return sens
        
        
        


The optimal value is -1.0286880469444515
A solution z is
[ 1.62457413 -0.61152765]
A dual solution is
[8.69658363e-07 1.91243046e-07]


In [38]:
import torch
import numpy as np

# assigns weights to data points, currently all 1's just to test other functions
def w(x, y, model):
    return torch.ones(len(y))

# samples minibatch
def get_batch(x, y, batch_size):
    inds = np.random.choice(range(len(y)), size=batch_size, replace=False)
    return x[inds], y[inds]
    
# performs an iteration of the iteratively reweighted least squares algorithm
def step(x, y, model, optimizer, w, batch_size):
    # arguments are self explanatory except w needs to be a callable function, i used a lambda below in the test
    #crit = torch.nn.MSELoss()
    optimizer.zero_grad()
    x, y = get_batch(x, y, batch_size)
    y_hat = model(x)
    weights = w(x, y, model)
    loss = torch.norm((y_hat - y))**2/batch_size
    #loss = crit(y, y_hat)
    loss.backward(retain_graph=True)
    optimizer.step()
    return loss

# gets parameters of model as array, used to check convergence later
def get_params(model):
    params=[]
    param_generator = model.parameters()
    for param in param_generator:
        nums = param.flatten()
        for num in nums:
            params.append(num.item())
    return np.array(params)

# training loop that stops when parameters dont change much, we can explore other convergence criteria
def train(x, y, model, optimizer, w, batch_size, eps):
    done = False
    params = get_params(model)
    losses=[]
    i=0
    while not done:
        i+=1
        print(i)
        loss = step(x, y, model, optimizer, w, batch_size)
        losses.append(loss)
        new_params = get_params(model)
        if np.linalg.norm(params-new_params)<eps:
            done = True
        else:
            params = new_params
    return losses   

In [39]:
# test training loop on a linear least squares problem with no noise, looks like its working

n=100 # number of obervations
p = 3 # dimension of context variable x or number of predictors
batch_size = 100
eps = 1e-3 # convergence criteria
sigma = .03

#m = 2 # dimension of decision variable z


true_model = torch.nn.Sequential(
      torch.nn.Linear(in_features=p, out_features=m)
)
true_params = get_params(true_model)
x = torch.randn(n, p)
y = true_model(x) + sigma*torch.randn(n,m)

In [40]:
# train sensitivity model
model = torch.nn.Sequential(
      torch.nn.Linear(in_features=p, out_features=m)
)
start_params = get_params(model)
optimizer = torch.optim.Adam(model.parameters())
#yo = lambda x, c, batch_size: w(x, c, batch_size) # this is the callable function to calculate weights
losses = train(x, y, model, optimizer, fd, batch_size, eps)
end_params = get_params(model)
print("true params are", true_params)
print("found params are", end_params)
print("relative errror is", np.linalg.norm(end_params-true_params)/np.linalg.norm(true_params))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [41]:
# train sensitivity model
model_mle = torch.nn.Sequential(
      torch.nn.Linear(in_features=p, out_features=m)
)
start_params_mle = get_params(model_mle)
optimizer_mle = torch.optim.Adam(model_mle.parameters())
#yo = lambda x, c, batch_size: w(x, c, batch_size) # this is the callable function to calculate weights
losses = train(x, y, model_mle, optimizer_mle, w, batch_size, eps)
end_params_mle = get_params(model_mle)
print("true params are", true_params)
print("found params are", end_params_mle)
print("relative errror is", np.linalg.norm(end_params_mle-true_params)/np.linalg.norm(true_params))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [42]:
costs = torch.zeros(len(x))
costs_mle = torch.zeros(len(x))
best_costs = torch.zeros(len(x))
x_test = torch.randn(n, p)
y_test = (true_model(x) + sigma*torch.randn(n,m)).detach().numpy() 

y_hat = model(x_test).detach().numpy()
y_hat_mle = model_mle(x_test).detach().numpy()


for i in range(len(y_hat)):
    
    # get sens decision
    z = cp.Variable(m)
    prob = cp.Problem(cp.Minimize(y_hat[i,:].T@z),
            [b0 <= z, z <= b1])
    prob.solve()
    z0 = z.value
    
    # get cost of sens decision
    costs[i] = y_test[i,:].T@z0
    
    
    # get mle decision
    z = cp.Variable(m)
    prob = cp.Problem(cp.Minimize(y_hat_mle[i,:].T@z),
            [b0 <= z, z <= b1])
    prob.solve()
    z0 = z.value
    
    # get cost of mle decision
    costs_mle[i] = y_test[i,:].T@z0
    
    
    
    # get cost of decision using knowledge of actual y
    z = cp.Variable(m)
    prob = cp.Problem(cp.Minimize(y_test[i,:].T@z),
            [b0 <= z, z <= b1])
    prob.solve()
    best_costs[i] = prob.value
    
    
    
    

In [43]:
torch.sum(costs_mle - costs > 0)

tensor(24)

In [44]:
torch.sum(costs_mle == costs )

tensor(59)

In [45]:
torch.mean(costs_mle - costs)

tensor(7.7961e-06)

In [46]:
torch.mean(costs - best_costs)

tensor(0.0001)

In [47]:
torch.mean(best_costs)

tensor(0.4025)

In [48]:
torch.mean(costs_mle)

tensor(0.4027)

In [49]:
torch.mean(costs)

tensor(0.4027)

In [50]:
costs - best_costs

tensor([ 1.1658e-04,  8.3163e-05,  1.3332e-04,  4.1729e-04,  1.3566e-04,
         3.8683e-05,  1.8841e-04,  3.1942e-04,  7.6286e-05,  6.9463e-04,
         1.4901e-08,  6.7592e-05,  0.0000e+00,  6.7139e-04,  0.0000e+00,
         0.0000e+00,  1.1921e-07,  1.0788e-04,  4.1455e-05,  0.0000e+00,
         8.4996e-04,  2.2089e-04,  6.2895e-04,  1.9923e-04,  2.6563e-04,
         0.0000e+00,  0.0000e+00,  4.7296e-05,  0.0000e+00,  1.8001e-04,
         3.0696e-05,  1.3739e-05,  3.9537e-04,  4.3797e-04,  1.3530e-05,
         5.9605e-08, -5.9605e-08,  0.0000e+00,  1.8671e-05,  2.2024e-05,
         1.2170e-04,  3.4635e-04,  1.5408e-05,  3.3379e-04,  2.1970e-04,
         1.6272e-04,  6.2576e-04,  5.7995e-04,  0.0000e+00,  0.0000e+00,
         7.2896e-05,  0.0000e+00,  8.1241e-05,  0.0000e+00,  1.2672e-04,
         0.0000e+00,  0.0000e+00,  6.4993e-04,  7.4506e-09,  3.2017e-04,
         0.0000e+00,  0.0000e+00,  9.6500e-05,  0.0000e+00,  1.2130e-05,
         2.9607e-05,  2.1648e-04,  0.0000e+00,  1.9

In [51]:
y_hat

array([[ 0.03370203, -0.5087079 ],
       [ 0.6315022 , -0.79729986],
       [ 0.15124871, -0.05153872],
       [ 0.15881442,  1.0418583 ],
       [-0.48158592,  0.59677684],
       [ 0.3029006 , -0.3961468 ],
       [-0.49232805,  0.01178443],
       [-0.35821807, -0.6986855 ],
       [ 0.03153027, -0.33688593],
       [-0.8824157 , -2.186666  ],
       [-0.0992422 , -0.10808671],
       [ 0.8851172 ,  0.83503234],
       [-0.2642398 , -1.6534435 ],
       [ 0.90093356,  0.2835026 ],
       [ 0.73802507,  0.88928354],
       [ 0.25898555, -0.6401271 ],
       [ 0.30478117,  0.02318847],
       [ 0.67720556, -0.5240947 ],
       [-0.7356768 , -1.2167082 ],
       [ 0.51627225,  0.28243038],
       [-0.06759741, -0.9849981 ],
       [-0.09935812,  0.57741296],
       [ 0.77926975,  0.11958888],
       [ 0.54867285,  0.3259667 ],
       [ 1.5924739 ,  1.3988702 ],
       [ 0.7611498 ,  0.12149194],
       [-0.47603112, -0.745144  ],
       [ 0.33167988, -0.15801299],
       [ 0.0610337 ,

In [52]:
y_hat_mle

array([[ 0.1098031 , -0.39395112],
       [-0.07982174, -0.30623806],
       [ 0.196294  ,  0.00546086],
       [ 0.9755006 ,  1.1195891 ],
       [ 0.75686073,  0.32680938],
       [ 0.08093268, -0.17519374],
       [ 0.37424332, -0.25374258],
       [ 0.16191643, -0.72244287],
       [ 0.11433715, -0.2886962 ],
       [-0.5760628 , -2.3726232 ],
       [ 0.3286156 , -0.10834436],
       [ 0.5968927 ,  1.2280483 ],
       [-0.48144612, -1.6115453 ],
       [ 0.31325117,  0.741105  ],
       [ 0.7115768 ,  1.2354009 ],
       [-0.02971859, -0.41371125],
       [ 0.28674623,  0.19801897],
       [-0.08800188, -0.117185  ],
       [-0.10405919, -1.4232388 ],
       [ 0.3467902 ,  0.5207914 ],
       [-0.16607419, -0.90364   ],
       [ 0.6084765 ,  0.46726793],
       [ 0.16095015,  0.4788301 ],
       [ 0.2861414 ,  0.53242797],
       [ 0.77177405,  2.1091602 ],
       [ 0.22837856,  0.5074618 ],
       [ 0.06578864, -0.87896395],
       [ 0.22107711,  0.06732616],
       [ 0.40933698,